# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
print(f"geoapify_key {geoapify_key}") 

geoapify_key a8e8780a741e4dd6b1f361142bc0b1dc


In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    #tiles = True,
    tiles = "EsriImagery",
    #tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map plot
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [144]:
# Narrow down cities that fit criteria and drop any results with null values
good_cities_df  = city_data_df[(city_data_df["Max Temp"] >= 5.0) &\
(city_data_df["Max Temp"] <= 15.0) & (city_data_df["Humidity"] < 65)\
& (city_data_df["Cloudiness"] < 20)]

# Drop any rows with null values
good_cities_cleaned_df = good_cities_df.dropna(how="any")

# Display sample data
good_cities_cleaned_df.shape
good_cities_cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
44,44,collierville,35.0420,-89.6645,7.11,47,0,5.66,US,1666108151
90,90,saint george,37.1041,-113.5841,13.33,44,0,1.54,US,1666108286
96,96,zhangye,38.9342,100.4517,11.11,16,0,1.67,CN,1666108292
115,115,cedar city,37.6775,-113.0619,14.11,49,0,0.00,US,1666108304
141,141,brenham,30.1669,-96.3977,14.47,40,0,3.09,US,1666108319
158,158,almaznyy,48.0448,40.0450,10.39,47,15,3.44,RU,1666108332
174,174,avera,33.1940,-82.5271,12.53,26,0,4.90,US,1666108342
176,176,challapata,-18.9000,-66.7667,14.15,30,0,5.26,BO,1666108343
231,231,carrollton,32.9537,-96.8903,14.63,31,0,5.66,US,1666108361


### Step 3: Create a new DataFrame called `hotel_df`.

In [156]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = good_cities_cleaned_df.copy()
hotel_df = hotel_df[["City","Lat","Lng","Humidity","Country"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
2,new norfolk,-42.7826,147.0587,58,AU,
44,collierville,35.0420,-89.6645,47,US,
90,saint george,37.1041,-113.5841,44,US,
96,zhangye,38.9342,100.4517,16,CN,
115,cedar city,37.6775,-113.0619,49,US,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [174]:
# Set parameters to search for a hotel
radius = 10000
limit = 20

params = {
    "filter": '',
    "limit":limit,
    "bias": '',
    "categories": 'accommodation.hotel',
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

hotel_df = hotel_df.reset_index(drop=True)
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lat = hotel_df.iloc[index, 1]
    lng = hotel_df.iloc[index, 2]
    #print(f"lat: {lat} lng: {lng}")
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    print(f"base_url: {base_url}")
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
base_url: https://api.geoapify.com/v2/places
params {'filter': 'circle:147.0587,-42.7826,10000', 'limit': 20, 'bias': 'proximity:147.0587,-42.7826', 'categories': 'accommodation.hotel', 'apiKey': 'a8e8780a741e4dd6b1f361142bc0b1dc'}
new norfolk - nearest hotel: Woodbridge on the Derwent
base_url: https://api.geoapify.com/v2/places
params {'filter': 'circle:-89.6645,35.042,10000', 'limit': 20, 'bias': 'proximity:-89.6645,35.042', 'categories': 'accommodation.hotel', 'apiKey': 'a8e8780a741e4dd6b1f361142bc0b1dc'}
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
base_url: https://api.geoapify.com/v2/places
params {'filter': 'circle:-113.5841,37.1041,10000', 'limit': 20, 'bias': 'proximity:-113.5841,37.1041', 'categories': 'accommodation.hotel', 'apiKey': 'a8e8780a741e4dd6b1f361142bc0b1dc'}
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
base_url: https://api.geoapify.com/v2/places
params {'filter': 'circl

,City,Lat,Lng,Humidity,Country,Hotel Name
0,new norfolk,-42.7826,147.0587,58,AU,Woodbridge on the Derwent
1,collierville,35.0420,-89.6645,47,US,Days Inn & Suites by Wyndham Collierville Germ...
2,saint george,37.1041,-113.5841,44,US,"The Advenire, an Autograph Collection Hotel"
3,zhangye,38.9342,100.4517,16,CN,Zhangye Hotel 张掖饭店
4,cedar city,37.6775,-113.0619,49,US,Baymont Inn & Suites
5,brenham,30.1669,-96.3977,40,US,Ant Street Inn
6,almaznyy,48.0448,40.0450,47,RU,Зверево
7,avera,33.1940,-82.5271,26,US,No hotel found
8,challapata,-18.9000,-66.7667,30,BO,Residencial Marian
9,carrollton,32.9537,-96.8903,31,US,Days Inn North Dallas/Farmers Branch


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [187]:
%%capture --no-display
# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    #tiles = True,
    tiles = "EsriImagery",
    #tiles = "EsriImagery",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.7,
    color = "City",
    hover_cols=["Hotel Name"]
)

# Display the map plot
map_plot_3
# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)